In [8]:
import numpy as np
import scipy.stats as stats
import unittest


In [76]:


class BlockingSystemSimulation:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            # Generate interarrival time and update arrival time
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]
            

            # Check for available service units
            if len(self.service_end_times) < self.m:
                # Generate service time
                service_time = np.random.exponential(self.mean_service_time)
                self.service_times.append(service_time)
                # Service can start immediately
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                # All service units are busy, customer is blocked
                self.blocked_customers += 1

            
           
    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        print(se)
        return p - z * se, p + z * se

if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list =[]
    for i in range(10+1):
        simulation = BlockingSystemSimulation(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = simulation.fraction_blocked()
    ci_lower, ci_upper = simulation.confidence_interval()

    print(f"Fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.1259
Simulation 2: Blocked customers = 0.1191
Simulation 3: Blocked customers = 0.1117
Simulation 4: Blocked customers = 0.1264
Simulation 5: Blocked customers = 0.1290
Simulation 6: Blocked customers = 0.1198
Simulation 7: Blocked customers = 0.1224
Simulation 8: Blocked customers = 0.1224
Simulation 9: Blocked customers = 0.1235
Simulation 10: Blocked customers = 0.1213
Simulation 11: Blocked customers = 0.1241
0.0032969560203314815
Fraction of blocked customers: 0.1241
95% Confidence interval for blocked fraction: (0.1176, 0.1306)


In [37]:
import numpy as np
import scipy.stats as stats

def confidence_interval_t(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    std_err = stats.sem(data)  # Standard error of the mean
    t_crit = stats.t.ppf((1 + confidence) / 2, df=n-1)
    h = t_crit * std_err
    return mean - h, mean + h

# Example usage:
block_list = block_list # Replace with your simulation results
ci_lower, ci_upper = confidence_interval_t(block_list)
print(f"95% t-confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

95% t-confidence interval for mean blocked fraction: (0.1547, 0.1614)


In [36]:
def erlang_b(m, T):
    """
    Compute the Erlang B blocking probability for m servers and offered traffic T.
    """
    from math import factorial

    numerator = (T ** m) / factorial(m)
    denominator = sum((T ** i) / factorial(i) for i in range(m + 1))
    return numerator / denominator

# Example usage:
m = 10
T = 8  # offered traffic in Erlangs
blocking_probability = erlang_b(m, T)
print(f"Erlang B blocking probability (m={m}, T={T}): {blocking_probability:.4f}")

Erlang B blocking probability (m=10, T=8): 0.1217


question 2

In [77]:


class BlockingSystemSimulation_2:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []
    


    def simulate(self, erlang_k=2):
        lmbda = erlang_k
        arrival_time = 0

        for _ in range(self.num_customers):
            # Always use Erlang (Gamma) interarrival times
            interarrival_time = np.random.gamma(shape=erlang_k, scale=1/lmbda)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Generate service time
            service_time = np.random.exponential(self.mean_service_time)
            self.service_times.append(service_time)

            # Remove completed services
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            # Check for available service units
            if len(self.service_end_times) < self.m:
                # Service can start immediately
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                # All service units are busy, customer is blocked
                self.blocked_customers += 1
    

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se

if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list =[]
    k = 8  # or any integer > 1 for Erlang-k
    lmbda = k  # so mean = k / lmbda = 1
    
    for i in range(10):
        simulation = BlockingSystemSimulation_2(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate(k)
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")
    

    blocked_fraction = simulation.fraction_blocked()
    ci_lower, ci_upper = simulation.confidence_interval()

    print(f"Fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.0658
Simulation 2: Blocked customers = 0.0621
Simulation 3: Blocked customers = 0.0689
Simulation 4: Blocked customers = 0.0678
Simulation 5: Blocked customers = 0.0703
Simulation 6: Blocked customers = 0.0698
Simulation 7: Blocked customers = 0.0725
Simulation 8: Blocked customers = 0.0686
Simulation 9: Blocked customers = 0.0739
Simulation 10: Blocked customers = 0.0675
Fraction of blocked customers: 0.0675
95% Confidence interval for blocked fraction: (0.0626, 0.0724)


In [78]:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    for k in range(1, 11):  # k from 1 to 10
        lmbda = k
        block_list = []
        for i in range(10):
            simulation = BlockingSystemSimulation_2(m, mean_service_time, mean_interarrival_time, num_customers)
            simulation.simulate(erlang_k=k)
            block_list.append(simulation.fraction_blocked())
        ci_lower, ci_upper = confidence_interval_t(block_list)
        sum_blocked = sum(block_list)
        mean_blocked = np.mean(block_list)
        print(f"k={k}: Mean blocked fraction = {mean_blocked:.4f}, 95% CI = ({ci_lower:.4f}, {ci_upper:.4f})")

k=1: Mean blocked fraction = 0.1201, 95% CI = (0.1154, 0.1247)
k=2: Mean blocked fraction = 0.0945, 95% CI = (0.0894, 0.0995)
k=3: Mean blocked fraction = 0.0820, 95% CI = (0.0777, 0.0862)
k=4: Mean blocked fraction = 0.0756, 95% CI = (0.0711, 0.0800)
k=5: Mean blocked fraction = 0.0729, 95% CI = (0.0696, 0.0763)
k=6: Mean blocked fraction = 0.0696, 95% CI = (0.0672, 0.0720)
k=7: Mean blocked fraction = 0.0703, 95% CI = (0.0682, 0.0723)
k=8: Mean blocked fraction = 0.0662, 95% CI = (0.0631, 0.0693)
k=9: Mean blocked fraction = 0.0654, 95% CI = (0.0633, 0.0675)
k=10: Mean blocked fraction = 0.0670, 95% CI = (0.0641, 0.0699)


question 2a

In [81]:
class BlockingSystemSimulation_2b:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []
    


    def simulate(self):
        arrival_time = 0
        p1, lmbda1 = 0.8, 0.8333
        p2, lmbda2 = 0.2, 5.0
        for _ in range(self.num_customers):
            # Generate hyper-exponential interarrival time
            if np.random.rand() < p1:
                interarrival_time = np.random.exponential(1 / lmbda1)
            else:
                interarrival_time = np.random.exponential(1 / lmbda2)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            # Check for available service units
            if len(self.service_end_times) < self.m:
                service_time = np.random.exponential(self.mean_service_time)
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1
    

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se

In [83]:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_2b(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate()  # No arguments needed for hyper-exponential
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    # Use t-confidence interval for the mean of block_list
    ci_lower, ci_upper = confidence_interval_t(block_list)

    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.1287
Simulation 2: Blocked customers = 0.1325
Simulation 3: Blocked customers = 0.1386
Simulation 4: Blocked customers = 0.1387
Simulation 5: Blocked customers = 0.1481
Simulation 6: Blocked customers = 0.1375
Simulation 7: Blocked customers = 0.1417
Simulation 8: Blocked customers = 0.1410
Simulation 9: Blocked customers = 0.1422
Simulation 10: Blocked customers = 0.1340
Mean fraction of blocked customers: 0.1383
95% Confidence interval for mean blocked fraction: (0.1343, 0.1423)


Question 3a

In [84]:
class BlockingSystemSimulation_3a:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            # Poisson arrivals (exponential interarrival times)
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services before checking for available units
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            # Check for available service units
            if len(self.service_end_times) < self.m:
                # Constant service time
                service_time = self.mean_service_time
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se

In [86]:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_3a(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    ci_lower, ci_upper = confidence_interval_t(block_list)
    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.1275
Simulation 2: Blocked customers = 0.1230
Simulation 3: Blocked customers = 0.1226
Simulation 4: Blocked customers = 0.1216
Simulation 5: Blocked customers = 0.1199
Simulation 6: Blocked customers = 0.1168
Simulation 7: Blocked customers = 0.1235
Simulation 8: Blocked customers = 0.1271
Simulation 9: Blocked customers = 0.1202
Simulation 10: Blocked customers = 0.1249
Mean fraction of blocked customers: 0.1227
95% Confidence interval for mean blocked fraction: (0.1203, 0.1251)


3b

In [87]:
class BlockingSystemSimulation_Pareto:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers, pareto_k):
        self.m = m
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.pareto_k = pareto_k
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []

        # Calculate scale xm so that mean matches mean_service_time
        self.pareto_xm = mean_service_time * (pareto_k - 1) / pareto_k

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services before checking for available units
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            if len(self.service_end_times) < self.m:
                # Pareto service time
                service_time = np.random.pareto(self.pareto_k) * self.pareto_xm + self.pareto_xm
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se


In [88]:

# Example usage for k = 1.05 and k = 2.05
for pareto_k in [1.05, 2.05]:
    print(f"\nPareto service times with k = {pareto_k}")
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_Pareto(m, mean_service_time, mean_interarrival_time, num_customers, pareto_k)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    ci_lower, ci_upper = confidence_interval_t(block_list)
    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")


Pareto service times with k = 1.05
Simulation 1: Blocked customers = 0.0003
Simulation 2: Blocked customers = 0.0013
Simulation 3: Blocked customers = 0.0006
Simulation 4: Blocked customers = 0.0003
Simulation 5: Blocked customers = 0.0005
Simulation 6: Blocked customers = 0.0024
Simulation 7: Blocked customers = 0.0001
Simulation 8: Blocked customers = 0.0003
Simulation 9: Blocked customers = 0.0006
Simulation 10: Blocked customers = 0.0001
Mean fraction of blocked customers: 0.0006
95% Confidence interval for mean blocked fraction: (0.0001, 0.0012)

Pareto service times with k = 2.05
Simulation 1: Blocked customers = 0.1192
Simulation 2: Blocked customers = 0.1355
Simulation 3: Blocked customers = 0.1164
Simulation 4: Blocked customers = 0.1189
Simulation 5: Blocked customers = 0.1202
Simulation 6: Blocked customers = 0.1254
Simulation 7: Blocked customers = 0.1249
Simulation 8: Blocked customers = 0.1234
Simulation 9: Blocked customers = 0.1156
Simulation 10: Blocked customers = 0.

3c


In [89]:
class BlockingSystemSimulation_LogNormal:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers, sigma=1.0):
        self.m = m
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.sigma = sigma  # shape parameter for log-normal
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []

        # Calculate mu so that mean matches mean_service_time
        # mean = exp(mu + sigma^2 / 2) => mu = log(mean) - sigma^2 / 2
        self.mu = np.log(mean_service_time) - (sigma**2) / 2

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services before checking for available units
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            if len(self.service_end_times) < self.m:
                # Log-normal service time
                service_time = np.random.lognormal(self.mu, self.sigma)
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se


In [90]:

# Example usage:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    sigma = 1.0  # You can try other values for more/less variability
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_LogNormal(m, mean_service_time, mean_interarrival_time, num_customers, sigma)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    ci_lower, ci_upper = confidence_interval_t(block_list)
    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.1380
Simulation 2: Blocked customers = 0.1293
Simulation 3: Blocked customers = 0.1277
Simulation 4: Blocked customers = 0.1164
Simulation 5: Blocked customers = 0.1301
Simulation 6: Blocked customers = 0.1236
Simulation 7: Blocked customers = 0.1203
Simulation 8: Blocked customers = 0.1227
Simulation 9: Blocked customers = 0.1291
Simulation 10: Blocked customers = 0.1175
Mean fraction of blocked customers: 0.1255
95% Confidence interval for mean blocked fraction: (0.1207, 0.1302)
